# Building a Chatbot
This chatbot will will be able to have a conversion and remeber previous interactions


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')
  

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [6]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is Aneesh i am a 2nd year student")])

AIMessage(content="Hi Aneesh, it's nice to meet you! \n\nWhat can I do for you today? Are you working on any projects or have any questions I can help you with? I'm happy to chat or help in any way I can. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 24, 'total_tokens': 82, 'completion_time': 0.105454545, 'prompt_time': 0.002383585, 'queue_time': 0.28829814, 'total_time': 0.10783813}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b84386ed-8543-474f-a8db-bc5fcf200bb0-0', usage_metadata={'input_tokens': 24, 'output_tokens': 58, 'total_tokens': 82})

In [ ]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, my name is Aneesh i am a 2nd year student"),
    AIMessage(content="Hi Aneesh, it's nice to meet you! \n\nWhat can I do for you today? Are you working on any projects or have any questions I can help you with? I'm happy to chat or help in any way I can. 😊  \n\n"),
    HumanMessage(content="Hey Whats my name and what year am i from?")
])

AIMessage(content="You told me your name is Aneesh and that you are a 2nd year student.  😊  \n\nIs there anything else you'd like to know or talk about?  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 100, 'total_tokens': 141, 'completion_time': 0.074545455, 'prompt_time': 0.007079334, 'queue_time': 0.265185195, 'total_time': 0.081624789}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ae768a95-ffe9-4fc7-bb42-d9bb4ec6eeb6-0', usage_metadata={'input_tokens': 100, 'output_tokens': 41, 'total_tokens': 141})

### Message History
Message History class is used to wrap our model and make it more stateful. This will keep track of inputs and outputs of the model, and store them in some database. Future interactions will then load those messages and pass them into the chain as part of the input.

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [10]:
config = {"configurable":{"session_id":"chat1"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Aneesh i am a 2nd year student")],
    config=config
)

In [11]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content='Your name is Aneesh!  I remember from our first conversation.  😄 \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 118, 'total_tokens': 139, 'completion_time': 0.038181818, 'prompt_time': 0.00573121, 'queue_time': 0.231446369, 'total_time': 0.043913028}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ad8c9af-a19a-49e0-9fa3-7d53fa9de417-0', usage_metadata={'input_tokens': 118, 'output_tokens': 21, 'total_tokens': 139})

In [13]:
# change the config --> session_id
config1 = {"configurable":{"session_id":"chat2"}}
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)

AIMessage(content='As an AI, I have no memory of past conversations and do not know your name. Please tell me your name so I can address you properly.\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 15, 'total_tokens': 48, 'completion_time': 0.06, 'prompt_time': 0.001957346, 'queue_time': 0.23590793300000001, 'total_time': 0.061957346}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fbffbdbd-91d8-424e-bd51-5da3761ee06b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 33, 'total_tokens': 48})